In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from loader import Loader
from constant import FeatureType, FileType
from copy import deepcopy
from skimage.measure import compare_nrmse
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input, LSTM, Dense, RepeatVector
import time
import argparse
import numpy as np
import tensorflow as tf
import hashlib
import json

### Train, Validate, Test

In [3]:
def encoder_decoder(partition, args):
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']

    batch_size = args.batch_size
    epochs = args.epochs
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=args.early_stop)

    with tf.device('/GPU:0'):        
        encoder_inputs = Input(shape=(args.x_frames * 24, len(args.features)))

        encoder = LSTM(args.hid_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]

        repeat_vector = RepeatVector(24)
        repeat_vector_outputs = repeat_vector(encoder_outputs)

        decoder_lstm = LSTM(args.hid_dim, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(repeat_vector_outputs) # , initial_state=encoder_states)

        decoder_dense = Dense(1, activation=args.activation)
        decoder_outputs = decoder_dense(decoder_outputs)

        rmsprop = RMSprop(lr=args.lr)

        model = Model(encoder_inputs, decoder_outputs)
        model.compile(optimizer=rmsprop, loss=tf.keras.losses.MeanSquaredError(), metrics='accuracy')

        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
              batch_size=batch_size, epochs=epochs, callbacks=[callback])
        
        scores = model.evaluate(X_val, y_val, verbose=0)
        
        print('LSTM validation scores:', scores)
        print(model.summary())

    return model, history

In [4]:
def test(partition, model, args):
    X_test, y_test = partition['test']
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape(y_test.shape[0], y_test.shape[1])

    nrmse = compare_nrmse(y_test, y_pred)
    return y_pred, nrmse

In [5]:
def experiment(args):
    loader = Loader(args)
    partition = loader.get_dataset()
    
    ts = time.time()
    model, history = encoder_decoder(partition, args)
    te = time.time()
    
    print('Took {:2.2f} sec for training the model'.format(te-ts))
    
    y_pred, test_acc = test(partition, model, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = history.history['loss']
    result['val_losses'] = history.history['val_loss']
    result['train_accs'] = history.history['accuracy']
    result['val_accs'] = history.history['val_accuracy']
    result['train_acc'] = np.mean(history.history['accuracy'])
    result['val_acc'] = np.mean(history.history['val_accuracy'])
    result['test_nrmse'] = test_acc
    result['y_test'] = np.array_str(partition['test'][1])
    result['y_pred'] = np.array_str(y_pred)
    
    setattr(args, 'features', str(args.features))
    return vars(args), model, result

### Manage Experiment

In [6]:
def get_filepath(setting, filetype):
    exp_name = setting['exp_name']
    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    
    file_format = ""
    directory = ""
    if filetype == FileType.MODEL:
        file_format = 'h5'
        directory = 'models'
    elif filetype == FileType.RESULT:
        file_format = 'json'
        directory = 'results'
        
    filename = "%s-%s.%s" % (exp_name, hash_key, file_format)
    filepath = os.path.join(Path(os.getcwd()).parent.parent, directory, filename)
    return filepath
        
def save_exp_model(setting, model):
    filepath = get_filepath(setting, FileType.MODEL)
    model.save(filepath)

def save_exp_result(setting, result):
    filepath = get_filepath(setting, FileType.RESULT)
    
    result.update(setting)
    with open(filepath, 'w') as f:
        json.dump(result, f)
        
def load_exp_result(exp_name):
    dir_path = os.path.join(Path(os.getcwd()).parent.parent, "results")
    filenames = [f for f in listdeir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result)
    return df

In [7]:
# ====== Random Seed Initialization ====== #
seed = 1234
tf.random.set_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "solar power prediction using weather features"

# ====== Data Loading ====== #
args.plant = 126
args.spot = 174
args.start_date = "20190820"
args.end_date = "20200809"
args.batch_size = 64
args.x_frames = 3
args.y_frames = 1

# ====== Model Capacity ===== #
args.hid_dim = 256

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop'
args.activation = 'relu'
args.lss = 'MSE'
args.lr = 0.001
args.epochs = 256
args.early_stop = 30
args.evaluation = 'NRMSE'

# ====== Experiment Variable ====== #
name_var1 = 'features'
list_var1 = [FeatureType.SUNSHINE,
                 FeatureType.HUMIDITY,
                 FeatureType.WIND_SPEED,
                 FeatureType.VISIBILITY,
                 FeatureType.GROUND_TEMPERATURE,
                 FeatureType.WIND_DIRECTION,
                 FeatureType.STEAM_PRESSURE,
                 FeatureType.TEMPERATURE,
                 FeatureType.PRECIPITATION,
                 FeatureType.DEW_POINT_TEMPERATURE,
                 FeatureType.ATMOSPHERIC_PRESSURE]

for i in range(len(list_var1)):
    sub_list = list_var1[:i + 1]

    setattr(args, name_var1, str(sub_list))
    print(args)

    setattr(args, name_var1, sub_list)   
    setting, model, result = experiment(deepcopy(args))
    save_exp_model(setting, model)
    save_exp_result(setting, result)

Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)


/home/vislab/.conda/envs/tf-gpu/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Epoch 1/256
5/5 [==============================] - 1s 105ms/step - loss: 1316901120.0000 - accuracy: 0.2248 - val_loss: 855035904.0000 - val_accuracy: 0.0000e+00
Epoch 2/256
5/5 [==============================] - 0s 12ms/step - loss: 1316421120.0000 - accuracy: 0.0000e+00 - val_loss: 854912128.0000 - val_accuracy: 0.0000e+00
Epoch 3/256
5/5 [==============================] - 0s 11ms/step - loss: 1316329856.0000 - accuracy: 0.0000e+00 - val_loss: 854851776.0000 - val_accuracy: 0.0000e+00
Epoch 4/256
5/5 [==============================] - 0s 12ms/step - loss: 1316273664.0000 - accuracy: 0.0000e+00 - val_loss: 854802816.0000 - val_accuracy: 0.0000e+00
Epoch 5/256
5/5 [==============================] - 0s 12ms/step - loss: 1316225280.0000 - accuracy: 0.0000e+00 - val_loss: 854753152.0000 - val_accuracy: 0.0000e+00
Epoch 6/256
5/5 [==============================] - 0s 12ms/step - loss: 1316176768.0000 - accuracy: 0.0000e+00 - val_loss: 854707072.0000 - val_accuracy: 0.0000e+00
Epoch 7/256
5

5/5 [==============================] - 0s 10ms/step - loss: 1314286080.0000 - accuracy: 0.0000e+00 - val_loss: 852806528.0000 - val_accuracy: 0.0000e+00
Epoch 51/256
5/5 [==============================] - 0s 10ms/step - loss: 1314242176.0000 - accuracy: 0.0000e+00 - val_loss: 852765056.0000 - val_accuracy: 0.0000e+00
Epoch 52/256
5/5 [==============================] - 0s 11ms/step - loss: 1314200576.0000 - accuracy: 0.0000e+00 - val_loss: 852722240.0000 - val_accuracy: 0.0000e+00
Epoch 53/256
5/5 [==============================] - 0s 11ms/step - loss: 1314157952.0000 - accuracy: 0.0000e+00 - val_loss: 852675520.0000 - val_accuracy: 0.0000e+00
Epoch 54/256
5/5 [==============================] - 0s 11ms/step - loss: 1314112000.0000 - accuracy: 0.0000e+00 - val_loss: 852633920.0000 - val_accuracy: 0.0000e+00
Epoch 55/256
5/5 [==============================] - 0s 10ms/step - loss: 1314070784.0000 - accuracy: 0.0000e+00 - val_loss: 852590784.0000 - val_accuracy: 0.0000e+00
Epoch 56/256
5/5 

5/5 [==============================] - 0s 11ms/step - loss: 1312159616.0000 - accuracy: 0.0000e+00 - val_loss: 850668672.0000 - val_accuracy: 0.0000e+00
Epoch 100/256
5/5 [==============================] - 0s 10ms/step - loss: 1312117248.0000 - accuracy: 0.0000e+00 - val_loss: 850626432.0000 - val_accuracy: 0.0000e+00
Epoch 101/256
5/5 [==============================] - 0s 11ms/step - loss: 1312074752.0000 - accuracy: 0.0000e+00 - val_loss: 850580736.0000 - val_accuracy: 0.0000e+00
Epoch 102/256
5/5 [==============================] - 0s 11ms/step - loss: 1312029440.0000 - accuracy: 0.0000e+00 - val_loss: 850538816.0000 - val_accuracy: 0.0000e+00
Epoch 103/256
5/5 [==============================] - 0s 11ms/step - loss: 1311987584.0000 - accuracy: 0.0000e+00 - val_loss: 850495104.0000 - val_accuracy: 0.0000e+00
Epoch 104/256
5/5 [==============================] - 0s 12ms/step - loss: 1311944320.0000 - accuracy: 0.0000e+00 - val_loss: 850452160.0000 - val_accuracy: 0.0000e+00
Epoch 105/25

...

5/5 [==============================] - 0s 11ms/step - loss: 1313621120.0000 - accuracy: 0.0000e+00 - val_loss: 852141888.0000 - val_accuracy: 0.0000e+00
Epoch 65/256
5/5 [==============================] - 0s 12ms/step - loss: 1313581184.0000 - accuracy: 0.0000e+00 - val_loss: 852100928.0000 - val_accuracy: 0.0000e+00
Epoch 66/256
5/5 [==============================] - 0s 12ms/step - loss: 1313539840.0000 - accuracy: 0.0000e+00 - val_loss: 852059072.0000 - val_accuracy: 0.0000e+00
Epoch 67/256
5/5 [==============================] - 0s 12ms/step - loss: 1313519744.0000 - accuracy: 0.0000e+00 - val_loss: 852025472.0000 - val_accuracy: 0.0000e+00
Epoch 68/256
5/5 [==============================] - 0s 11ms/step - loss: 1313457664.0000 - accuracy: 0.0000e+00 - val_loss: 851972672.0000 - val_accuracy: 0.0000e+00
Epoch 69/256
5/5 [==============================] - 0s 12ms/step - loss: 1313411968.0000 - accuracy: 0.0000e+00 - val_loss: 851925568.0000 - val_accuracy: 0.0000e+00
Epoch 70/256
5/5 

5/5 [==============================] - 0s 11ms/step - loss: 1311507840.0000 - accuracy: 0.0000e+00 - val_loss: 850014016.0000 - val_accuracy: 0.0000e+00
Epoch 114/256
5/5 [==============================] - 0s 11ms/step - loss: 1311465344.0000 - accuracy: 0.0000e+00 - val_loss: 849972736.0000 - val_accuracy: 0.0000e+00
Epoch 115/256
5/5 [==============================] - 0s 11ms/step - loss: 1311423744.0000 - accuracy: 0.0000e+00 - val_loss: 849928960.0000 - val_accuracy: 0.0000e+00
Epoch 116/256
5/5 [==============================] - 0s 11ms/step - loss: 1311380224.0000 - accuracy: 0.0000e+00 - val_loss: 849884800.0000 - val_accuracy: 0.0000e+00
Epoch 117/256
5/5 [==============================] - 0s 12ms/step - loss: 1311351040.0000 - accuracy: 0.0000e+00 - val_loss: 849864384.0000 - val_accuracy: 0.0000e+00
Epoch 118/256
5/5 [==============================] - 0s 11ms/step - loss: 1311319680.0000 - accuracy: 0.0000e+00 - val_loss: 849798976.0000 - val_accuracy: 0.0000e+00
Epoch 119/25

5/5 [==============================] - 0s 11ms/step - loss: 1309393408.0000 - accuracy: 0.0000e+00 - val_loss: 847886144.0000 - val_accuracy: 0.0000e+00
Epoch 163/256
5/5 [==============================] - 0s 11ms/step - loss: 1309347968.0000 - accuracy: 0.0000e+00 - val_loss: 847843456.0000 - val_accuracy: 0.0000e+00
Epoch 164/256
5/5 [==============================] - 0s 12ms/step - loss: 1309303936.0000 - accuracy: 0.0000e+00 - val_loss: 847820032.0000 - val_accuracy: 0.0000e+00
Epoch 165/256
5/5 [==============================] - 0s 11ms/step - loss: 1309258752.0000 - accuracy: 0.0000e+00 - val_loss: 847754560.0000 - val_accuracy: 0.0000e+00
Epoch 166/256
5/5 [==============================] - 0s 11ms/step - loss: 1309219712.0000 - accuracy: 0.0000e+00 - val_loss: 847707648.0000 - val_accuracy: 0.0000e+00
Epoch 167/256
5/5 [==============================] - 0s 11ms/step - loss: 1309171584.0000 - accuracy: 0.0000e+00 - val_loss: 847687104.0000 - val_accuracy: 0.0000e+00
Epoch 168/25

5/5 [==============================] - 0s 12ms/step - loss: 1307236864.0000 - accuracy: 0.0269 - val_loss: 845730880.0000 - val_accuracy: 0.0417
Epoch 213/256
5/5 [==============================] - 0s 11ms/step - loss: 1307198208.0000 - accuracy: 0.0305 - val_loss: 845691008.0000 - val_accuracy: 0.0417
Epoch 214/256
5/5 [==============================] - 0s 11ms/step - loss: 1307157632.0000 - accuracy: 0.0308 - val_loss: 845650624.0000 - val_accuracy: 0.0417
Epoch 215/256
5/5 [==============================] - 0s 11ms/step - loss: 1307118080.0000 - accuracy: 0.0300 - val_loss: 845600256.0000 - val_accuracy: 0.0417
Epoch 216/256
5/5 [==============================] - 0s 11ms/step - loss: 1307074816.0000 - accuracy: 0.0245 - val_loss: 845570432.0000 - val_accuracy: 0.0714
Epoch 217/256
5/5 [==============================] - 0s 11ms/step - loss: 1307030016.0000 - accuracy: 0.0406 - val_loss: 845532032.0000 - val_accuracy: 0.0417
Epoch 218/256
5/5 [==============================] - 0s 11ms

Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>, <FeatureType.HUMIDITY: '습도(%)'>, <FeatureType.WIND_SPEED: '풍속(m/s)'>, <FeatureType.VISIBILITY: '시정(10m)'>, <FeatureType.GROUND_TEMPERATURE: '지면온도(°C)'>, <FeatureType.WIND_DIRECTION: '풍향(16방위)'>, <FeatureType.STEAM_PRESSURE: '현지기압(hPa)'>, <FeatureType.TEMPERATURE: '기온(°C)'>, <FeatureType.PRECIPITATION: '강수량(mm)'>, <FeatureType.DEW_POINT_TEMPERATURE: '이슬점온도(°C)'>, <FeatureType.ATMOSPHERIC_PRESSURE: '증기압(hPa)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)
Epoch 1/256
5/5 [==============================] - 0s 95ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 2/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accu